# Amazon Bedrockで実践してみるRAG-Fusion

[RAG-Fusion](https://arxiv.org/abs/2402.03367)はご存じですか？  
これは既存のRAGの発展形として、複数のクエリとRRFを行使することでより包括的な回答を提供する試みです。  
今回はこのRAG-FusionをAmazon Bedrockを使って簡単に実装してみたいと思います。

## 前提条件
- Bedrock上でモデルのリクエストが完了している
- Bedrock上でナレッジベースができあがっている
- Bedrockの操作に必要なIAMが用意できている

## 準備
まずは必要なライブラリ等を準備します

In [91]:
import boto3
import json

In [92]:
# ナレッジベースのID
knowledge_base_id = ""
# データソースのID
data_source_id = ""
kb_client_runtime = boto3.client('bedrock-agent-runtime')
bedrock_runtime = boto3.client(service_name='bedrock-runtime')

# 推論用モデル(今回はClaudeのv2を使います)
model_id = "anthropic.claude-v2"
# クエリ拡張用のモデル(Claude Instantを使います。これは簡単なタスクにおける速度向上が狙いです)
q_model_id = "anthropic.claude-instant-v1"

region = "us-east-1"

## 推論の準備
Claudeで推論するための準備をします。  
今回はクエリ拡張と実際の推論時でモデルを使い分けたいので、引数に`model_id`を入れています。

In [94]:
def invoke_claude(text, model_id, max_tokens_to_sample=1000):
    body = json.dumps({
        "prompt": f"\n\nHuman:{text}\n\nAssistant: ",
        "max_tokens_to_sample": max_tokens_to_sample,
        "temperature": 0.1,
        "top_p": 0.9,
    })
    accept = 'application/json'
    content_type = 'application/json'

    response = bedrock_runtime.invoke_model(
        body=body,
        modelId=model_id,
        accept=accept,
        contentType=content_type
    )

    response_body = json.loads(response.get('body').read())
    return response_body.get('completion')[1:]

## クエリ拡張
次に、クエリの拡張用のコードを用意します。  
RAG-Fusionは元のユーザクエリに対して関連する検索クエリを複数生成させ、それらのクエリによるRAGの結果もあわせてランキング評価する試みです。  
ここではまず元のクエリに関連する検索クエリを生成させる部分を実装します。  
この際、**単調な類語変換ではなく、なるべく多角的な視点からクエリを生成させることがRAG-Fusionのコツ**となっています。

In [96]:
def generate_queries(original_query, n=4):
    f = ''
    for i in range(n):
        f += f'{n}: \n'

    prompt = f'''
以下に示すQueryはユーザの入力した検索クエリです。
このクエリに関連するクエリを多角的な視点から{n}つ生成してください
Formatに従って結果を出力してください

<Query>
{original_query}
</Query>

<Format>
{f}
</Format>
'''
    result = invoke_claude(prompt, q_model_id)
    result = result.split('<Format>')[1].split('</Format>')[0]
    # print(result)
    generated_queries = []
    for q in result.split('\n'):
        if q == '':
            continue
        generated_queries.append(q.split(' ')[1])
    return generated_queries[1:-1]

## RAG
続いてRAGの実装をします。  
今回はAmazon Bedrockのナレッジベースを使っているので非常に簡単に実行できます。

In [97]:
def kb_search(query, n=5):
    res = kb_client_runtime.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=knowledge_base_id,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': n
            }
        }
    )

    # {doc: score}の形に整形します
    return_dict = {}
    for r in res['retrievalResults']:
        return_dict[r['content']['text']] = r['score']

    return return_dict

## RRF
次にRRFを実装します。  
複数クエリによるRAGの結果を集約して再評価している部分です。

In [98]:
def reciprocal_rank_fusion(all_results, k=50):
    fused_scores = {}
    for query, doc_scores in all_results.items():
        for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)):
            if doc not in fused_scores:
                fused_scores[doc] = 0
            previous_score = fused_scores[doc]
            fused_scores[doc] += 1 / (rank + k)

    reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
    return reranked_results

## 実行
一通りの準備ができたので実行していきます。  

まずは検索クエリを拡張します。

In [99]:
original_query = '確定申告について教えて'

queries = generate_queries(original_query)
queries.insert(0, original_query)

続いて、拡張されたクエリ群を使って通常のRAG検索をします。

In [101]:
all_results = {}
for q in queries:
    all_results[q] = kb_search(q)

RRFで再評価します。

In [103]:
# ここでは上位5件を採用している
reranked_results = list(reciprocal_rank_fusion(all_results).keys())[:5]

RAG-Fusionの結果を取り込みながらプロンプトを作成します。

In [104]:
information = ''
for i, r in enumerate(reranked_results):
    information += f'情報{i+1}. {r}\n'

prompt = """
日本では本日(2月16日)から確定申告の相談及び申告書の受付がスタートしましたね。
この国の風物詩であるこの確定申告という儀式では、毎年数多の迷える子羊が出現することで有名です。
あなたは全知全能の税神として、迷える子羊の抱いている確定申告に対する悩みを聞き、彼らを正しき納税へと導いてください。
全知全能たるあなたのために、Informationセクションには迷える子羊の悩みに関連しそうな情報を提供します。有効に活用してください。

<Information>
{information}
</Information>

<UserQuery>
{query}
</UserQuery>
"""
prompt = prompt.format(information=information, query=original_query)

回答を生成します。

In [105]:
print(invoke_claude(prompt, model_id))

確定申告について、以下の点をご説明します。

- 確定申告は、1年間の所得や税額を確定させるための申告です。所得税や住民税の申告書を税務署に提出することになります。

- 確定申告が必要な方は、給与所得者で年末調整を受けていない方や、自営業者の方などです。申告が必要かどうかは情報1の「確定申告の概要」をご確認ください。

- 申告期間は2月16日から3月15日までです。郵送やe-Taxで提出できます。提出方法は情報1の「申告書の提出方法」をご覧ください。

- 申告書には収入や所得控除などを正確に記入します。所得税と住民税で取り扱いが異なる項目があるので、住民税の記入も必要です。

- 申告書には本人確認のための書類を添付する必要があります。情報5をご確認ください。

- 納税は3月15日が期限です。納付方法は情報2をご覧ください。

確定申告で不明な点があれば、ご質問ください。ご案内できる範囲でお答えします。


# Example
## 通常のRAGによる回答
```
確定申告について、以下の点をご説明します。

1. 確定申告の概要
- 所得税や復興特別所得税を納める義務がある人は、1年間の所得について確定申告を行う必要があります。
- 確定申告は、申告書に必要事項を記入して税務署に提出することで行います。

2. 確定申告が必要な人
- 給与所得者でも、一定の条件を満たす場合は確定申告が必要です。
- 自営業者や農業者などの申告が必要な所得がある人も確定申告が必要です。 

3. 確定申告のメリット
- 税額控除などを受けることができ、納める税金を減らすことができます。
- 医療費控除などにより、所得税や住民税の還付を受けることができます。

4. 確定申告の手続き
- 確定申告書に必要事項を記入し、添付書類を添えて税務署に提出します。
- e-Taxなどの電子申告も利用できます。
- 申告期限は原則3月15日です。

5. 納税
- 確定申告後、計算した税額を納付期限までに納める必要があります。 
- 納付には納税通知書や振替納税などの方法があります。

確定申告の概要は以上の通りです。不明な点があればご質問ください。
```

## RAG-Fusionによる回答
```
確定申告に関しては、以下の点に注意が必要です。

1. 確定申告の手続き
- 申告期間は2月16日から3月15日までです。郵送またはe-Taxで提出できます。
- 申告書の控えに収受印をもらう場合は、郵送時に返信用封筒を同封するか、受付で控えを持参する必要があります。

2. 納税の方法
- 納付期限は3月15日です。キャッシュレス納付が便利です。
- 振替納税を利用する場合は3月15日までに手続きが必要です。

3. 控除の記入
- 医療費控除、社会保険料控除、生命保険料控除などの控除を受ける場合は、第一表と第二表にそれぞれ記入が必要です。

4. 住民税・事業税
- 別居家族の氏名・住所を記入する必要があります。

5. 還付金の受取
- 公金受取口座を登録・利用する場合は手続きが必要です。

以上の点に注意し、確定申告の手続きを進めましょう。不明な点があれば税務署に問い合わせることをおすすめします。
```

内容が正しいかどうかはともかく、RAG-Fusionの方が還付金等、情報の幅が広がっているように見えます。